# Selenium Tips and Tricks

## Manage your Webdrivers with webdriver-manager

When using `Selenium` with Python, you probably did the following:

- Download Chromedriver Binary
- Unzip it
- Set the path to the driver

This is annoying.

- The Path can be changed
- You have to somehow manage those browser drivers for each OS
- Check if new updates for drivers are released

Instead of doing this manually, use `webdriver-manager`.

It makes managing binaries for different browsers easy.

`webdriver-manager` downloads binaries automatically for you.

So you don't have to go through the pain of doing it manually.


To use it in your project, see the example below. It's straightforward and saves you time and energy.

Especially when you integrate `Selenium` in your CI/CD Pipeline.

By default, `webdriver-manager` installs the latest version.

But you can also define a specific version of the driver.

In [ ]:
!pip install webdriver-manager

In [ ]:
# Old way
from selenium import webdriver
driver = webdriver.Chrome('path/to/driver.exe')

# New way with webdriver-manager and Selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# New way with webdriver-manager and Selenium 3
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

# Use specific version
driver = webdriver.Chrome(executable_path=ChromeDriverManager("<your_version>").install())